In [126]:
import pandas as pd
import numpy as np

# Load the data from CSV files
dim_data = pd.read_csv('dim.csv')
fact_data = pd.read_csv('fact.csv')

# Define the constants for transaction codes
PAYMENT_TYPE_CODE = 'OPTP0028'  # Code for payment transactions
SPENDING_TYPE_CODE = 'OPTP0000'  # Code for spending transactions

# Convert transaction time to datetime object for easier manipulation
fact_data['TXN_TM'] = pd.to_datetime(fact_data['TXN_TM'])

# Sort the transactions by time to ensure the order is correct for subsequent operations
fact_data_sorted = fact_data.sort_values(by='TXN_TM', ignore_index=True)

# Filter out invalid transaction
fact_data_sorted = fact_data_sorted.loc[~fact_data_sorted['TXN_SRC_ID'].isna()]

# Helper function to get the statement period based on a transaction date
def get_statement_period(txn_date):
    # print(txn_date)
    if txn_date.day <= 22:
        # Statement period starts on the 22nd of the previous month and ends on the 21st of the transaction month
        statement_start = txn_date.replace(day=22, hour=00, minute=00, second=00) - pd.DateOffset(months=1)
        statement_end = txn_date.replace(day=21, hour=00, minute=00, second=00)
    else:
        statement_start = txn_date.replace(day=22, hour=00, minute=00, second=00) 
        statement_end = txn_date.replace(day=21, hour=00, minute=00, second=00) + pd.DateOffset(months=1)
    return statement_start, statement_end

# Helper function to calculate the due date of payment
def get_payment_due_date(txn_date):
    # Payment due date is the 5th of the month following the transaction month
    due_date = txn_date.replace(day=5, hour=00, minute=00, second=00) + pd.DateOffset(months=1)
    return due_date

fact_data_sorted.loc[fact_data_sorted['TML_WEB_AP_NM'] == SPENDING_TYPE_CODE, "NET_CASH_FLOW_AMT_LCY"] *= -1

# Add a column for the statement period start and end, and the payment due date to the payments dataframe
# payments = fact_data_sorted[fact_data_sorted['TML_WEB_AP_NM'] == PAYMENT_TYPE_CODE].copy()
# fact_data_sorted[['STATEMENT_START', 'STATEMENT_END']] = fact_data_sorted['TXN_TM'].apply(
#     lambda x: pd.Series(get_statement_period(x))
# )
fact_data_sorted['STATEMENT_START'], fact_data_sorted['STATEMENT_END'] = zip(
    *fact_data_sorted['TXN_TM'].apply(get_statement_period)
)
fact_data_sorted['PAYMENT_DUE_DATE'] = fact_data_sorted['TXN_TM'].apply(get_payment_due_date)


In [127]:
fact_data_sorted[fact_data_sorted['TML_WEB_AP_NM'] == PAYMENT_TYPE_CODE].head(3)

,CIF hash,CARD_NBR hash,TXN_TM,TML_WEB_AP_NM,DSC,NET_CASH_FLOW_AMT_LCY,TXN_SRC_ID,CARD_CLASSCIFICATION,STATEMENT_START,STATEMENT_END,PAYMENT_DUE_DATE
88,815e4858d422f45f27ff703fce8acfed,af446dd0dce35b7c0d687b32466726c89239b37dd8fcf9...,2022-05-31 21:23:35,OPTP0028,TT QUA TPBANK EBANKING 44 LE NGOC HAN HA NOI V...,1520565.0,272976284.0,VC,2022-05-22,2022-06-21,2022-06-05
91,73842a366de67e8d76320590e6a6ced8,5001c0d5c425bdfdcd5108671045068a43e3012dc4faf7...,2022-06-01 12:54:21,OPTP0028,TT QUA TPBANK EBANKING 44 LE NGOC HAN HA NOI V...,3500000.0,273092292.0,VC,2022-05-22,2022-06-21,2022-07-05
118,0828e14ddfd5dcfe9b2fc7a54eeba5f8,f0b80252f29a51ae72a1e4a259ca6857824f00e62cc868...,2022-06-06 15:57:12,OPTP0028,TT QUA TPBANK EBANKING 44 LE NGOC HAN HA NOI V...,143912.0,274393715.0,VC,2022-05-22,2022-06-21,2022-07-05


In [128]:
# filter type transaction
filtered_data = fact_data_sorted.loc[(fact_data_sorted['TML_WEB_AP_NM'] == PAYMENT_TYPE_CODE) | (fact_data_sorted['TML_WEB_AP_NM'] == SPENDING_TYPE_CODE)]

In [129]:
# filtered_data['ACCUM_BALANCE'] = filtered_data.groupby(['CIF hash', 'CARD_NBR hash', 'STATEMENT_START', 'STATEMENT_END'])['NET_CASH_FLOW_AMT_LCY'].cumsum()
filtered_data['ACCUM_BALANCE'] = filtered_data.groupby(['CIF hash', 'CARD_NBR hash'])['NET_CASH_FLOW_AMT_LCY'].cumsum()

/tmp/ipykernel_2128007/3833195438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['ACCUM_BALANCE'] = filtered_data.groupby(['CIF hash', 'CARD_NBR hash'])['NET_CASH_FLOW_AMT_LCY'].cumsum()


In [130]:
filtered_data = filtered_data.merge(dim_data, on=['CIF hash', 'CARD_NBR hash'])


In [131]:
filtered_data.groupby(['CIF hash', 'CARD_NBR hash', 'STATEMENT_START', 'STATEMENT_END', 'PAYMENT_DUE_DATE']).agg({'TXN_TM': 'max'})

TXN_TM
CIF hash                         CARD_NBR hash                                      STATEMENT_START STATEMENT_END PAYMENT_DUE_DATE                    
0340ab3837f34a1aa87d5b5a8a25a07e 8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0aa... 2022-08-22      2022-09-21    2022-10-05       2022-09-21 18:14:36
                                                                                    2022-09-22      2022-10-21    2022-10-05       2022-09-27 13:53:45
                                                                                    2022-10-22      2022-11-21    2022-12-05       2022-11-22 11:16:44
                                                                                    2022-11-22      2022-12-21    2023-01-05       2022-12-08 08:06:55
                                                                                    2022-12-22      2023-01-21    2023-02-05       2023-01-03 15:28:38
...                                                                                                                                                ...
fabbc6f82889b39ea2529bcd3134ede2 a8ea8bc7dd239166636d3626765c69e5a056c57b0ffd1e1... 2023-07-22      2023-08-21    2023-09-05       2023-08-08 10:03:16
fb57dffb7f96a780b33ba00ba0f2d8c3 a466359e5b3742757bbbf27f9e33386f0f0c746fd6bdc8d... 2023-05-22      2023-06-21    2023-07-05       2023-06-13 21:11:36
                                                                                    2023-06-22      2023-07-21    2023-08-05       2023-07-21 09:54:19
                                                                                    2023-07-22      2023-08-21    2023-08-05       2023-07-31 20:55:31
                                                                                                                  2023-09-05       2023-08-18 20:27:08

[1407 rows x 1 columns]

In [132]:
filtered_data[filtered_data['CIF hash']=='0340ab3837f34a1aa87d5b5a8a25a07e'].head(8)

,CIF hash,CARD_NBR hash,TXN_TM,TML_WEB_AP_NM,DSC,NET_CASH_FLOW_AMT_LCY,TXN_SRC_ID,CARD_CLASSCIFICATION,STATEMENT_START,STATEMENT_END,PAYMENT_DUE_DATE,ACCUM_BALANCE,CREDIT_LIMIT
5462,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-09-19 21:52:59,OPTP0000,CTYVIMO*HOKINHDO 0824853333 704,-15728000.0,303376660.0,VC,2022-08-22,2022-09-21,2022-10-05,-15728000.0,17000000
5463,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-09-21 18:13:42,OPTP0000,Foody 19002042 704,-1000.0,303934425.0,VC,2022-08-22,2022-09-21,2022-10-05,-15729000.0,17000000
5464,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-09-21 18:13:45,OPTP0000,Foody 19002042 704,-1000.0,303934425.0,VC,2022-08-22,2022-09-21,2022-10-05,-15730000.0,17000000
5465,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-09-21 18:14:36,OPTP0000,Foody 19002042 704,-78000.0,303934767.0,VC,2022-08-22,2022-09-21,2022-10-05,-15808000.0,17000000
5466,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-09-27 13:53:45,OPTP0000,Foody 19002042 704,-253000.0,305578899.0,VC,2022-09-22,2022-10-21,2022-10-05,-16061000.0,17000000
5467,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-11-04 17:57:35,OPTP0028,TT QUA TPBANK EBANKING 44 LE NGOC HAN HA NOI V...,16060000.0,316780911.0,VC,2022-10-22,2022-11-21,2022-12-05,-1000.0,17000000
5468,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-11-04 17:58:46,OPTP0000,TPBANK QPAY 57 LY THUONG KIET HA NOI VNM 704 0...,-16000000.0,316781459.0,VC,2022-10-22,2022-11-21,2022-12-05,-16001000.0,17000000
5469,0340ab3837f34a1aa87d5b5a8a25a07e,8a310933f4b7f835b19a0a8a2ab43ef865e49111d3fb0a...,2022-11-11 11:24:05,OPTP0000,Foody 19002042 704,-103000.0,318841726.0,VC,2022-10-22,2022-11-21,2022-12-05,-16104000.0,17000000


In [189]:
filtered_data['TXN_TM'] = pd.to_datetime(filtered_data['TXN_TM'])
suspicious_transaction_case2 = {}
for cif_hash in fact_data_sorted['CIF hash'].values:
    t = filtered_data[filtered_data['CIF hash']==cif_hash]
    empty_balance_time = {}
    # supicious_transaction = {}
    for index, row in t.iterrows():
        if row['ACCUM_BALANCE'] >= -1000 and row['ACCUM_BALANCE'] <= 1000:
            empty_balance_time[row['TXN_TM']] = index

    for time, index in empty_balance_time.items():
        while filtered_data.loc[index, 'TXN_TM'] - time <= pd.Timedelta(minutes=30):
            if abs(filtered_data.loc[index, 'ACCUM_BALANCE']) >= 0.9*filtered_data.loc[index, 'CREDIT_LIMIT']:
                suspicious_transaction_case2[filtered_data.loc[index, 'CIF hash']]= index
                break
            index += 1
    
    # suspicious_transaction_case2[cif_hash] = supicious_transaction



In [99]:
t = filtered_data[filtered_data['CIF hash']=='0deb82c7391489aa02c771fec3b67f0f']
empty_balance_time = {}
supicious_transaction = {}
for index, row in t.iterrows():
    if row['ACCUM_BALANCE'] >= -1000 and row['ACCUM_BALANCE'] <= 1000:
        empty_balance_time[row['TXN_TM']] = index

for time, index in empty_balance_time.items():
    while filtered_data.loc[index, 'TXN_TM'] - time <= pd.Timedelta(minutes=30):
        if abs(filtered_data.loc[index, 'ACCUM_BALANCE']) >= 0.9*filtered_data.loc[index, 'CREDIT_LIMIT']:
            supicious_transaction[filtered_data.loc[index, 'CIF hash']]= index
            break
        index += 1
supicious_transaction

{}

In [184]:
last_transaction_per_statement_periods = filtered_data.groupby(['CIF hash', 'CARD_NBR hash', 'STATEMENT_START', 'STATEMENT_END', 'PAYMENT_DUE_DATE']).agg({'TXN_TM': 'max'})
suspicious_transaction_case1 = {}
for k, v in last_transaction_per_statement_periods.iterrows():
    cif_hash, _, statement_start, statement_end, due_date = k
    last_transaction_date = v['TXN_TM']
    filter_result = filtered_data[np.logical_and(filtered_data['CIF hash'] == cif_hash, 
                                     filtered_data['TXN_TM'] == last_transaction_date
                                     )]
    current_balance = filter_result['ACCUM_BALANCE'].values[0]
    credit_limit = filter_result['CREDIT_LIMIT'].values[0]
    if abs(current_balance) >= 0.9 * credit_limit:
        # print('balance', cif_hash, last_transaction_date, current_balance, credit_limit)
        if cif_hash in suspicious_transaction_case1:
            continue
        suspicious_transaction_case1[cif_hash] = True
        continue


balance 0340ab3837f34a1aa87d5b5a8a25a07e 2022-09-21 18:14:36 -15808000.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2022-09-27 13:53:45 -16061000.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2022-11-22 11:16:44 -16885600.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2022-12-08 08:06:55 -16929600.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2023-01-03 15:28:38 -16929600.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2023-02-28 11:56:59 -16929600.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2023-04-04 14:08:03 -16933884.0 17000000
balance 0340ab3837f34a1aa87d5b5a8a25a07e 2023-05-04 15:12:00 -16933884.0 17000000
balance 05a6d37e7ea9a73c5343618218d0ddc6 2023-02-17 16:32:39 -47016646.0 40000000
balance 05a6d37e7ea9a73c5343618218d0ddc6 2023-05-22 13:45:11 -51558797.0 40000000
balance 05a6d37e7ea9a73c5343618218d0ddc6 2023-05-28 16:04:05 -52348797.0 40000000
balance 05a6d37e7ea9a73c5343618218d0ddc6 2023-07-30 13:32:02 -49152463.0 40000000
balance 05a6d37e

In [186]:
len(suspicious_transaction_case1.keys())

52

In [193]:
supicious_list = set(suspicious_transaction_case2.keys()).union(set(suspicious_transaction_case1.keys()))

{'0340ab3837f34a1aa87d5b5a8a25a07e',
 '05a6d37e7ea9a73c5343618218d0ddc6',
 '0d1223a89432121b203d46615e62d802',
 '0d31729255847e0dd796888052c805f2',
 '0ed36ad419f7c4445dd50d4bf5c1f547',
 '0f442033bc13322abc628fb7f800ee3c',
 '116abb6fc1e47f1e49b0865f556ce8aa',
 '16c2f23828a7ee45a690c36650cf2913',
 '16ee64914e79a87a4abb5c5842ae5eb5',
 '17e330c86d7307fd7168bf567e39d367',
 '17f54211892dd53b7728a094799ef9db',
 '24dc185ddc00fdbe7a3519e5a3bb2c12',
 '26345247ea23ba0d58eba8853a75658d',
 '279db73b357d7e5ad31ac75693599bf1',
 '2cb3fc60ab6a9f230f115fa07eb7d686',
 '2ce7875e386eccfc7347c7539d3b5639',
 '2f4470cbc13df1712ef4973a457c1748',
 '2f95e5fec21e0fd3be38ea313484ac63',
 '4f5a39e1204e049da7edf68b8f5f3414',
 '59a749f0c763d8c3f30580c824d95fa1',
 '6234bb182f285f0e3c7d48840c8ba6c7',
 '6246181344bd77893a1ee8ba9408ec7d',
 '626ef5a51add74b6f85c885fc3063527',
 '62af7a290813a2231c1d29de6ede7843',
 '63666920a777c263ccb9f4de1b0b0e36',
 '70b5f453e222eb4a13e1b0bebebd36b5',
 '7e9c172d235c590cbe8eb74b1489b7c8',
 